In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
            print("Name:", gpu.name, "  Type:", gpu.device_type)

    except RuntimeError as e:
        print(e)

Name: /physical_device:GPU:0   Type: GPU


In [2]:
import pandas as pd
import numpy as np
import math 
from multiprocessing import cpu_count
from joblib import Parallel
from joblib import delayed
import matplotlib.pyplot as plt
from sklearn.utils import shuffle as skshuffle
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Input
from sklearn import preprocessing
from tensorflow.keras import backend as K
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
import pyswarms as ps
import random
from pylab import rcParams
%matplotlib inline

In [3]:
loss_tracker = keras.metrics.Mean(name="loss")
mae_metric = keras.metrics.MeanAbsoluteError(name="mae")


In [4]:
np.__version__

'1.18.5'

In [5]:
tf.__version__

'2.4.1'

In [6]:
rcParams['figure.figsize'] = 50, 10

In [7]:
PATH_TO_TEST_TRAIN_DATA = 'Tangni/Data/'

In [8]:
class LSTM_Tangni:
    def __init__(self,dictionary):
        
        self.train_size = 62
        
        
        self.data1 = np.array(pd.read_csv(PATH_TO_TEST_TRAIN_DATA + '1-3.csv',  header=0, usecols=[0, 1, 2, 3], index_col=None))
        self.data2 = np.array(pd.read_csv(PATH_TO_TEST_TRAIN_DATA + '2-12.csv', header=0, usecols=[0, 1, 2, 3], index_col=None))
        self.data3 = np.array(pd.read_csv(PATH_TO_TEST_TRAIN_DATA + '3-6.csv',  header=0, usecols=[0, 1, 2, 3], index_col=None))
        self.data4 = np.array(pd.read_csv(PATH_TO_TEST_TRAIN_DATA + '4-15.csv', header=0, usecols=[0, 1, 2, 3], index_col=None))
        self.data5 = np.array(pd.read_csv(PATH_TO_TEST_TRAIN_DATA + '5-15.csv', header=0, usecols=[0, 1, 2, 3], index_col=None))     
        
    def Print(self):
        print(self.data1)
        
    def rmse(self, true, predict):
        
        r = true - predict
        sq = r*r
        avg = np.mean(sq)
        error = math.sqrt(avg)

        return error
    
    def Average(self, lst):
        return sum(lst) / len(lst)
    
    def dataProcessing(self, dF, lookback):
    
        data=list()
        label=list()
        for i in range(len(dF) - lookback):

            data.append(np.array(dF[i:i+lookback,:4]))

            label.append(np.array(dF[i+lookback,1:2]))
        
        return np.array(data), np.array(label)

    def make_packets(self, lookback):
        
        X1, L1 = self.dataProcessing(self.data1, lookback)
        X2, L2 = self.dataProcessing(self.data2, lookback)
        X3, L3 = self.dataProcessing(self.data3, lookback)
        X4, L4 = self.dataProcessing(self.data4, lookback)
        X5, L5 = self.dataProcessing(self.data5, lookback)
        
        train_X1, label_X1,test_X1,tst_label_X1 = X1[:self.train_size - lookback], L1[:self.train_size - lookback], X1[self.train_size - lookback:], L1[self.train_size - lookback:]
        train_X2, label_X2,test_X2,tst_label_X2 = X2[:self.train_size - lookback], L2[:self.train_size - lookback], X2[self.train_size - lookback:], L2[self.train_size - lookback:]
        train_X3, label_X3,test_X3,tst_label_X3 = X3[:self.train_size - lookback], L3[:self.train_size - lookback], X3[self.train_size - lookback:], L3[self.train_size - lookback:]
        train_X4, label_X4,test_X4,tst_label_X4 = X4[:self.train_size - lookback], L4[:self.train_size - lookback], X4[self.train_size - lookback:], L4[self.train_size - lookback:]
        train_X5, label_X5,test_X5,tst_label_X5 = X5[:self.train_size - lookback], L5[:self.train_size - lookback], X5[self.train_size - lookback:], L5[self.train_size - lookback:]
        
    
        return train_X1, label_X1,test_X1,tst_label_X1, train_X2, label_X2,test_X2,tst_label_X2,train_X3, label_X3,test_X3,tst_label_X3, train_X4, label_X4,test_X4,tst_label_X4, train_X5, label_X5,test_X5,tst_label_X5

    def getData(self, lookback = 1, shuffle=0):
        
        train_X1, label_X1,test_X1,tst_label_X1, train_X2, label_X2,test_X2,tst_label_X2,train_X3, label_X3,test_X3,tst_label_X3, train_X4, label_X4,test_X4,tst_label_X4, train_X5, label_X5,test_X5,tst_label_X5 = self.make_packets(lookback)
        DATA = [train_X1, label_X1,test_X1,tst_label_X1, train_X2, label_X2,test_X2,tst_label_X2,train_X3, label_X3,test_X3,tst_label_X3, train_X4, label_X4,test_X4,tst_label_X4, train_X5, label_X5,test_X5,tst_label_X5]
        
        Train = np.append(train_X1, np.append(train_X2, np.append(train_X3, np.append(train_X4, train_X5, axis=0), axis=0), axis=0), axis=0)
        Label = np.append(label_X1, np.append(label_X2, np.append(label_X3, np.append(label_X4, label_X5, axis=0), axis=0), axis=0), axis=0) 
     
        if (shuffle == 1):
            Train, Label = skshuffle(Train, Label, random_state=0)
            
        return Train, Label
    
    def LSTM_model(self, units, lookback, features):
        
        model = Sequential()
        model.add(LSTM(units = units, return_sequences = False, activation= 'tanh', input_shape = (lookback,features)))

        model.add(Dense(units = 1, activation= 'linear'))
        model.compile(optimizer = 'adam', loss = 'mean_squared_error')
#         model.summary()
        
        return model
    
    def trainModel(self, units, lookback = 1, batch_size = 4, shuffle = 0):
        
        train_X1, label_X1,test_X1,tst_label_X1, train_X2, label_X2,test_X2,tst_label_X2,train_X3, label_X3,test_X3,tst_label_X3, train_X4, label_X4,test_X4,tst_label_X4, train_X5, label_X5,test_X5,tst_label_X5 = self.make_packets(lookback)
        DATA = [train_X1, label_X1,test_X1,tst_label_X1, train_X2, label_X2,test_X2,tst_label_X2,train_X3, label_X3,test_X3,tst_label_X3, train_X4, label_X4,test_X4,tst_label_X4, train_X5, label_X5,test_X5,tst_label_X5]
        
        Train = np.append(train_X1, np.append(train_X2, np.append(train_X3, np.append(train_X4, train_X5, axis=0), axis=0), axis=0), axis=0)
        Label = np.append(label_X1, np.append(label_X2, np.append(label_X3, np.append(label_X4, label_X5, axis=0), axis=0), axis=0), axis=0) 
     
        if (shuffle == 1):
            Train, Label = skshuffle(Train, Label, random_state=0)
            
        LSTM = self.LSTM_model(units, lookback)
        LSTM.fit(Train, Label, epochs = 300, batch_size = batch_size, verbose=0)
        
        RMSE = list()
        
        #Calculate the error of the traing and testing
        for i in range(0, 20, 2):
            
            prediction = LSTM.predict(DATA[i])
            RMSE.append(self.rmse(prediction, DATA[i+1]))
           
         
        #Find average of the training and testing
        RMSE.append(self.Average(RMSE[0:5]))
        RMSE.append(self.Average(RMSE[5:10]))
        
      
        return RMSE
    
    def runModel(self, units, features, lookback = 1, epochs=1, batch_size = 4, shuffle = 0):
        
        train_X1, label_X1,test_X1,tst_label_X1, train_X2, label_X2,test_X2,tst_label_X2,train_X3, label_X3,test_X3,tst_label_X3, train_X4, label_X4,test_X4,tst_label_X4, train_X5, label_X5,test_X5,tst_label_X5 = self.make_packets(lookback)
        DATA = [train_X1, label_X1,test_X1,tst_label_X1, train_X2, label_X2,test_X2,tst_label_X2,train_X3, label_X3,test_X3,tst_label_X3, train_X4, label_X4,test_X4,tst_label_X4, train_X5, label_X5,test_X5,tst_label_X5]
        
        Train = np.append(train_X1, np.append(train_X2, np.append(train_X3, np.append(train_X4, train_X5, axis=0), axis=0), axis=0), axis=0)
        Label = np.append(label_X1, np.append(label_X2, np.append(label_X3, np.append(label_X4, label_X5, axis=0), axis=0), axis=0), axis=0) 
     
        if (shuffle == 1):
            Train, Label = skshuffle(Train, Label, random_state=0)
        
        LSTM = self.LSTM_model(units, lookback, features)
        
        LSTM.fit(Train, Label, epochs = epochs, batch_size = batch_size, verbose=0)
        
        RMSE = []
        MAE = []
        MAPE = []
        P =[]
        Pre=[]
        T =[]
        sh = (1,lookback,features)
        
        #Calculate the error of the traing and testing
        
        A, B, C, D = self.accuracy(LSTM,train_X1,train_X1,label_X1,sh,r=0)
        RMSE.append(A),MAE.append(B),MAPE.append(C),Pre.append(D)
        
        A, B, C, D = self.accuracy(LSTM,train_X2,train_X2,label_X2,sh,r=0)
        RMSE.append(A),MAE.append(B),MAPE.append(C),Pre.append(D)
        
        A, B, C, D = self.accuracy(LSTM,train_X3,train_X3,label_X3,sh,r=0)
        RMSE.append(A),MAE.append(B),MAPE.append(C),Pre.append(D)
        
        A, B, C, D = self.accuracy(LSTM,train_X4,train_X4,label_X4,sh,r=0)
        RMSE.append(A),MAE.append(B),MAPE.append(C),Pre.append(D)
        
        A, B, C, D = self.accuracy(LSTM,train_X5,train_X5,label_X5,sh,r=0)
        RMSE.append(A),MAE.append(B),MAPE.append(C),Pre.append(D)
        
        A, B, C, D = self.accuracy(LSTM,train_X1,test_X1,tst_label_X1,sh,r=1)
        RMSE.append(A),MAE.append(B),MAPE.append(C),Pre.append(D)
        
        A, B, C, D = self.accuracy(LSTM,train_X2,test_X2,tst_label_X2,sh,r=1)
        RMSE.append(A),MAE.append(B),MAPE.append(C),Pre.append(D)
        
        A, B, C, D = self.accuracy(LSTM,train_X3,test_X3,tst_label_X3,sh,r=1)
        RMSE.append(A),MAE.append(B),MAPE.append(C),Pre.append(D)
        
        A, B, C, D = self.accuracy(LSTM,train_X4,test_X4,tst_label_X4,sh,r=1)
        RMSE.append(A),MAE.append(B),MAPE.append(C),Pre.append(D)
        
        A, B, C, D = self.accuracy(LSTM,train_X5,test_X5,tst_label_X5,sh,r=1)
        RMSE.append(A),MAE.append(B),MAPE.append(C),Pre.append(D)
        
        
#         #Calculate the error of the traing and testing
#         for i in range(0, 20, 2):
            
#             prediction = LSTM.predict(DATA[i])
#             RMSE.append(self.rmse(prediction, DATA[i+1]))
#             plt.plot(prediction )
#             plt.plot(DATA[i+1], color='r')
#             plt.xticks(fontsize=25)
#             plt.yticks(fontsize=25)
#             plt.ylim((-6,2))
#             plt.show()
         
#         #Find average of the training and testing
#         RMSE.append(self.Average(RMSE[0:5]))
#         RMSE.append(self.Average(RMSE[5:10]))
        
      
        return RMSE,MAE,MAPE,Pre
    
    def accuracy(self, LSTM, Data1, Data2, Data3, shape, r):
        
        P =[]
           
        if r==0:
            D1 = np.copy(Data1[0])
            D1=D1.reshape(shape)
            D2 = np.copy(Data2[1:])
            Testing = np.copy(Data3)
            for i in range(len(D2)):
                p=LSTM.predict(D1)
                P.append(p[-1])
                a=np.array(D2[i])
                a=a.reshape(shape)
                D1=np.append(D1,a,axis=0)
            P=np.array(P)
            RMSE = math.sqrt(mean_squared_error(Testing[1:], P))
            MAE  = mean_absolute_error(Testing[1:], P)
            MAPE = mean_absolute_percentage_error(Testing[1:], P)
            
        if r==1:
            D1 = np.copy(Data1)
            D2 = np.copy(Data2)
            Testing = np.copy(Data3)
            for i in range(16):
                p=LSTM.predict(D1)
                P.append(p[-1])
                a=np.array(D2[i])
                a=a.reshape(shape)
                D1=np.append(D1,a,axis=0)
            P=np.array(P)
            RMSE = math.sqrt(mean_squared_error(Testing, P))
            MAE  = mean_absolute_error(Testing, P)
            MAPE = mean_absolute_percentage_error(Testing, P)
            
        return RMSE, MAE, MAPE, P
        
    
    def runModel_weights(self, units, features, weights, lookback = 1, epochs=1, batch_size = 1, shuffle = 0):
        
        train_X1, label_X1,test_X1,tst_label_X1, train_X2, label_X2,test_X2,tst_label_X2,train_X3, label_X3,test_X3,tst_label_X3, train_X4, label_X4,test_X4,tst_label_X4, train_X5, label_X5,test_X5,tst_label_X5 = self.make_packets(lookback)
        DATA = [train_X1, label_X1,test_X1,tst_label_X1, train_X2, label_X2,test_X2,tst_label_X2,train_X3, label_X3,test_X3,tst_label_X3, train_X4, label_X4,test_X4,tst_label_X4, train_X5, label_X5,test_X5,tst_label_X5]
        
        Train = np.append(train_X1, np.append(train_X2, np.append(train_X3, np.append(train_X4, train_X5, axis=0), axis=0), axis=0), axis=0)
        Label = np.append(label_X1, np.append(label_X2, np.append(label_X3, np.append(label_X4, label_X5, axis=0), axis=0), axis=0), axis=0) 
     
        if (shuffle == 1):
            Train, Label = skshuffle(Train, Label, random_state=0)
        
        LSTM = self.LSTM_model(units, lookback, features)
        
        idx1 = features * 4 * units
        idx2 = idx1 + units * 4 * units
        idx3 = idx2 + 4 * units
        idx4 = idx3 + units
        idx5 = idx4 + 1

        W1 = weights[0:idx1].reshape((features,4 * units))

        W2 = weights[idx1:idx2].reshape((units,4 * units))

        W3 = weights[idx2:idx3].reshape((4 * units))

        W4 = weights[idx3:idx4].reshape((units,1))

        W5 = weights[idx4:idx5].reshape((1))

        W = [W1,W2,W3,W4,W5]


        LSTM.set_weights(W)
        
        LSTM.fit(Train, Label, epochs = epochs, batch_size = batch_size, verbose=0)
        
        RMSE = []
        MAE = []
        MAPE = []
        P =[]
        Pre=[]
        T =[]
        sh = (1,lookback,features)
        
        #Calculate the error of the traing and testing
        
        A, B, C, D = self.accuracy(LSTM,train_X1,train_X1,label_X1,sh,r=0)
        RMSE.append(A),MAE.append(B),MAPE.append(C),Pre.append(D)
        
        A, B, C, D = self.accuracy(LSTM,train_X2,train_X2,label_X2,sh,r=0)
        RMSE.append(A),MAE.append(B),MAPE.append(C),Pre.append(D)
        
        A, B, C, D = self.accuracy(LSTM,train_X3,train_X3,label_X3,sh,r=0)
        RMSE.append(A),MAE.append(B),MAPE.append(C),Pre.append(D)
        
        A, B, C, D = self.accuracy(LSTM,train_X4,train_X4,label_X4,sh,r=0)
        RMSE.append(A),MAE.append(B),MAPE.append(C),Pre.append(D)
        
        A, B, C, D = self.accuracy(LSTM,train_X5,train_X5,label_X5,sh,r=0)
        RMSE.append(A),MAE.append(B),MAPE.append(C),Pre.append(D)
        
        A, B, C, D = self.accuracy(LSTM,train_X1,test_X1,tst_label_X1,sh,r=1)
        RMSE.append(A),MAE.append(B),MAPE.append(C),Pre.append(D)
        
        A, B, C, D = self.accuracy(LSTM,train_X2,test_X2,tst_label_X2,sh,r=1)
        RMSE.append(A),MAE.append(B),MAPE.append(C),Pre.append(D)
        
        A, B, C, D = self.accuracy(LSTM,train_X3,test_X3,tst_label_X3,sh,r=1)
        RMSE.append(A),MAE.append(B),MAPE.append(C),Pre.append(D)
        
        A, B, C, D = self.accuracy(LSTM,train_X4,test_X4,tst_label_X4,sh,r=1)
        RMSE.append(A),MAE.append(B),MAPE.append(C),Pre.append(D)
        
        A, B, C, D = self.accuracy(LSTM,train_X5,test_X5,tst_label_X5,sh,r=1)
        RMSE.append(A),MAE.append(B),MAPE.append(C),Pre.append(D)
        
        RMSE.append(np.sum(RMSE[:5])/5)
        RMSE.append(np.sum(RMSE[5:])/5)
             
        
#         for i in range(len(P)):
#             print(Testing[i],P[i])
        
        
        return RMSE, MAE, MAPE, Pre
            

In [9]:
dictionary = {'normalize': 0}

In [10]:
lstm = LSTM_Tangni(dictionary)

In [11]:
class CustomModel(keras.Model):
    def train_step(self, data):

        x, y = data
        
        y_pred = self(x, training=True)
        loss = self.compiled_loss(y, y_pred, regularization_losses=self.losses)

         # Compute our own metrics
        loss_tracker.update_state(loss)
        self.compiled_metrics.update_state(y, y_pred)
        # Return a dict mapping metric names to current value
        return {m.name: m.result() for m in self.metrics}

    # Forward propagation
def forward_props(params, units, features):
    
    idx1 = features * 4 * units
    idx2 = idx1 + units * 4 * units
    idx3 = idx2 + 4 * units
    idx4 = idx3 + units
    idx5 = idx4 + 1
    
    W1 = params[0:idx1].reshape((features,4 * units))
   
    W2 = params[idx1:idx2].reshape((units,4 * units))
    
    W3 = params[idx2:idx3].reshape((4 * units))
    
    W4 = params[idx3:idx4].reshape((units,1))
    
    W5 = params[idx4:idx5].reshape((1))
    
    W = [W1,W2,W3,W4,W5]
   
    
    modelOne.set_weights(W)

    history = modelOne.fit(Train, Label, epochs = 1, batch_size = 1000, verbose=0)
    
    
    loss = history.history['loss'][0]

    return loss

def fun(x,print_step):

    n_particles = x.shape[0]
    j = [forward_props(x[i],units, features) for i in range(n_particles)]
    return np.array(j)

In [16]:
units = 200
features = 4
shuffle = 0
lookback = 3

Train, Label =lstm.getData(lookback=lookback, shuffle=shuffle)
inputs = keras.Input(shape=(lookback,features))
L = keras.layers.LSTM(units = units, activation='tanh')(inputs)
outputs = keras.layers.Dense(1, activation='linear')(L)
modelOne = CustomModel(inputs, outputs)
modelOne.compile(optimizer="adam", loss="mse", metrics=['mae'])

# Initialize swarm
options = {'c1': 0.5, 'c2': 0.3, 'w':0.9}

# Call instance of PSO

dimensions = features *(4 * units) + units *(4 * units) + (4 * units) + units + 1
optimizer = ps.single.GlobalBestPSO(n_particles=100, dimensions=dimensions, options=options)

# Perform optimization
cost, posfinal = optimizer.optimize(fun, print_step=100, iters=100, verbose=1)

2022-03-17 16:20:37,006 - pyswarms.single.global_best - INFO - Optimize for 100 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
pyswarms.single.global_best: 100%|█████████████████████|100/100, best_cost=0.337
2022-03-17 16:26:41,003 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 0.3368511199951172


##### 

In [40]:
RMSE,MAE,MAPE,Pre = lstm.runModel_weights(units=200, features=4, weights=posfinal, lookback = 3, epochs=500, batch_size = 64, shuffle = 0)


In [41]:
RMSE

[0.886766873720545,
 0.025760627236671014,
 0.044374272633781436,
 0.27880329307486723,
 0.744905325324031,
 0.15258482120527514,
 0.059871218426072896,
 1.153431683013035,
 0.7954974352365473,
 1.9094820904836947,
 0.39612207839797914,
 0.8933978653525207]

In [42]:
print(RMSE[0],RMSE[1],RMSE[2],RMSE[3],RMSE[4],RMSE[5],RMSE[6],RMSE[7],RMSE[8],RMSE[9])
print(MAE[0],MAE[1],MAE[2],MAE[3],MAE[4],MAE[5],MAE[6],MAE[7],MAE[8],MAE[9])
print(MAPE[0],MAPE[1],MAPE[2],MAPE[3],MAPE[4],MAPE[5],MAPE[6],MAPE[7],MAPE[8],MAPE[9])

0.886766873720545 0.025760627236671014 0.044374272633781436 0.27880329307486723 0.744905325324031 0.15258482120527514 0.059871218426072896 1.153431683013035 0.7954974352365473 1.9094820904836947
0.26494673400042673 0.020086152800198258 0.038334260970136765 0.07916399095006274 0.39033739573036785 0.11227299448698425 0.04161387962746427 0.6625758744982483 0.7340702039346925 1.6460396798774097
0.41574281018951526 0.008162062946633967 13.189804964147122 0.024174144390425714 0.6455845569940959 0.02029508507555618 0.01690991342899969 9.167577594021608 0.22295939315641508 1.2822100793596323


In [ ]:
# lookback = [1,2,3,4,5]
# unitss = [10, 50, 100, 150, 200, 300, 400, 500]
# Parameters = []
# Performance =[]
# minimum = 100
# idx = 0
# minidx = 0

# for i in lookback:
#         for u in unitss:
#             Parameters.append([u,i])
#             units = u
#             features = 4
#             Train, Label =lstm.getData(lookback=i, shuffle=0)

#             inputs = keras.Input(shape=(i,features))
#             L = keras.layers.LSTM(units = units, activation='tanh')(inputs)
#             outputs = keras.layers.Dense(1, activation='linear')(L)
#             modelOne = CustomModel(inputs, outputs)
#             modelOne.compile(optimizer="adam", loss="mse", metrics=['mae'])

#             # Initialize swarm
#             options = {'c1': 0.5, 'c2': 0.3, 'w':0.9}

#             # Call instance of PSO

#             dimensions = features *(4 * units) + units *(4 * units) + (4 * units) + units + 1
#             optimizer = ps.single.GlobalBestPSO(n_particles=100, dimensions=dimensions, options=options)

#             # Perform optimization
#             cost, pos = optimizer.optimize(fun, print_step=100, iters=100, verbose=1)

#             RMSE,MAE,MAPE,Pre = lstm.runModel_weights(units=u, features=4, weights=pos, lookback = i, epochs=500, batch_size = 64, shuffle = 0)

#             print([u,i],RMSE[-2],RMSE[-1])
#             Performance.append([u,i,RMSE[-2],RMSE[-1]])

#             if RMSE[-2] < minimum:
#                 minimum = RMSE[-2]
#                 minidx = idx

#             idx += 1
                
# print('--------------------------------------------------')
# print('Minimimu Error at [units, lookback, shuffle]')
# # print(Parameters[minidx])
# print(Performance[minidx])